# Directly Querying the ZTF (version 4) database
```Author: Eden Girma, Last updated 20210422```

# Table of contents:
* [Connecting to ZTF Database](#connecting)
* [Available database tables](#tables)
* [Example: Querying objects from past 24-48 hr](#ex1)
* [Exporting to VOTable](#exportVOTable)

**Goal:**
 
1) To query the ALeRCE database for objects with the following attributes:
* detected 24 - 48 hours from the current time
* classified by the stamp classifier (version 1.0.4)

2) To return a table consisting of ALeRCE alert objects that includes, per row:
* aggregated detection properties per object (e.g. mean RA/Dec, number of detections)
* probability of the highest ranking class assigned by the stamp classifier (v1.0.4)

In [1]:
import sys

# Packages for direct database access
# %pip install psycopg2
import psycopg2
import json

# Packages for data and number handling
import numpy as np
import pandas as pd
import math

# Packages for calculating current time and extracting ZTF data to VOTable
from astropy.time import Time
from astropy.table import Table
from astropy.io.votable import from_table, writeto
from datetime import datetime

# Packages for display and data plotting, if desired
from IPython.display import HTML
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline


## Connecting to ZTF Database - version 4 <a class="anchor" id="connecting"></a>

In [2]:
# Open and load credentials
credentials_file = "../alercereaduser_v4.json"
with open(credentials_file) as jsonfile:
    params = json.load(jsonfile)["params"]
    
# Open a connection to the database
conn = psycopg2.connect(dbname=params['dbname'], 
                        user=params['user'], 
                        host=params['host'], 
                        password=params['password'])

## Available database tables <a class="anchor" id="tables"></a>

The following cell shows all of the tables available in the database, which can be used for querying.

In [ ]:
# Show all the available tables, sorted by tablename
query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema='alerce'
ORDER BY table_name;
"""
tables = pd.read_sql_query(query, conn)
tables.sort_values(by="table_name")

The DB contains the following tables, among others:
    
* `object`: filter and bandpass aggregated properties per object
* `probability`: classification probabilities
* `magstat`: time aggregated bandpass dependent properties per object
* `ps1_ztf`: closest PanSTARRS object as reported by ZTF
* `ss_ztf`: closest MPC object as reported by ZTF
* `gaia_ztf`: closet GAIA source as reported by ZTF
* `detection`: individual detections, time and bandpass disaggregated
* `feature`: advanced object features, used for machine learning classification
* `non_detection`: limiting magnitudes in previous observations, the largest table of all

A more comprehensive list of the tables and their columns are available at: https://docs.google.com/spreadsheets/d/1OH3Dz-s8pWy-cY2FuT59miItHMrpIRm1aOVCRTPwLGA/edit#gid=974214313

Below, we can look at all of the columns that are available in all of the tables:

In [ ]:
query = """
SELECT table_name, column_name, data_type
FROM information_schema.columns
ORDER BY table_name;
"""
columns = pd.read_sql_query(query, conn)
display(columns[["table_name", "column_name", "data_type"]])
pd.options.display.max_rows = 101

## Querying objects from past 24-48 hr <a class="anchor" id="ex1"></a>

Let's pull all of the objects that have been classified between 24-48 hrs from now by the stamp classifier (version 1.0.4), along with their most likely probability class (ranking=1). 

In [3]:
min_lastmjd = Time(datetime.today(), scale='utc').mjd - 2
max_lastmjd = Time(datetime.today(), scale='utc').mjd - 1

query='''
SELECT
    object.oid, object.meanra, object.meandec, object.sigmara, object.sigmadec,
    object.firstmjd, object.lastmjd, object.ndet, 
    pr.classifier_name, pr.classifier_version, pr.class_name, 
    pr.ranking, pr.probability

FROM 
    object INNER JOIN (
        SELECT 
            probability.oid, probability.classifier_name, probability.classifier_version,
            probability.class_name, probability.ranking, probability.probability
        FROM
            probability
        WHERE
            probability.classifier_name = 'stamp_classifier'
            AND probability.classifier_version = 'stamp_classifier_1.0.4'
            AND probability.ranking = 1
    ) AS pr
    ON object.oid = pr.oid

WHERE 
    object.lastMJD >= %s
    AND object.lastMJD <= %s
''' % (min_lastmjd, max_lastmjd)

# Outputs as a pd.DataFrame
objects = pd.read_sql_query(query, conn)

In [21]:
# Prints the dataframe shape: (number of selected objects, number of selected filters)
print(objects.shape)

# Sorting detections by lastMJD in descending order
objects_sorted = objects.sort_values(by=['lastmjd', 'firstmjd', 'oid'], ascending=False)
objects_sorted.head()

(174548, 13)


,oid,meanra,meandec,sigmara,sigmadec,firstmjd,lastmjd,ndet,classifier_name,classifier_version,class_name,ranking,probability
82089,ZTF21aaxbzdg,351.739807,41.903393,NaN,NaN,59325.50875,59325.50875,1,stamp_classifier,stamp_classifier_1.0.4,bogus,1,0.610910
82085,ZTF21aaxbzdb,352.325332,40.726352,NaN,NaN,59325.50875,59325.50875,1,stamp_classifier,stamp_classifier_1.0.4,bogus,1,0.741378
82045,ZTF21aaxbzda,352.325603,40.726483,NaN,NaN,59325.50875,59325.50875,1,stamp_classifier,stamp_classifier_1.0.4,bogus,1,0.666561
82086,ZTF21aaxbzcz,352.325597,40.726480,NaN,NaN,59325.50875,59325.50875,1,stamp_classifier,stamp_classifier_1.0.4,bogus,1,0.665880
82007,ZTF21aaxbzcl,353.464751,37.546543,NaN,NaN,59325.50875,59325.50875,1,stamp_classifier,stamp_classifier_1.0.4,bogus,1,0.785700


In [17]:
# Count number of OIDs that correspond to each class name
obj_classes = objects.groupby('class_name')
for key in obj_classes.groups.keys():
    l = obj_classes.groups[key].size
    print('%s : %i' % (key, l))

AGN : 18564
SN : 1001
VS : 106670
asteroid : 26262
bogus : 22051


In [5]:
# Check if each row corresponds to a unique OID
objects['oid'].is_unique

True

## Exporting to VOTable <a class="anchor" id="exportVOTable"></a>

To save this data as a VOTable requires converting it from its current form (a ```pd.DataFrame```). This is possible with the ```Table``` object from ```astropy.table```, and the functions we initially imported from ```astropy.io.votable```. Essentially, we'll convert our ```pd.DataFrame``` to an ```astropy.table.Table``` to a ```astropy.io.votable.VOTableFile```, which can then be exported.

_A buggy caveat, however_ -- ```astropy.io.votable.VOTableFile``` objects throw an error when you attempt to pass on masked/```NaN``` values. I've gotten around this, for now, by filling in the masked values the the _string_ ```"None"``` before the ```pd.DataFrame``` is converted to a ```Table```.

In [22]:
# Filling the masked values with the string 'NaN'
objects_filled = objects_sorted.fillna('None')

# Converting filled dataframe to astropy Table, then astropy VOTableFile, then exporting into .xml
full_dt = Table.from_pandas(objects_filled)
votable = from_table(full_dt)
writeto(votable, "ztf_v4_DB_output.xml")